## **NOTE: Intead of using the downloaded datasets, I have used the EnergyFlow python package to directly download the datset into python .**

# PARTICLE-NET

In [70]:
%cd /content/

/content


In [71]:
!git clone https://github.com/hqucms/ParticleNet.git

Cloning into 'ParticleNet'...
remote: Enumerating objects: 20, done.
remote: Total 20 (delta 0), reused 0 (delta 0), pack-reused 20
Unpacking objects: 100% (20/20), done.


In [72]:
!pwd

/content


In [73]:
!pip install energyflow

In [74]:
import energyflow

In [75]:
data=energyflow.qg_jets.load(num_data=100000, pad=True, ncol=4, generator='pythia',with_bc=False, cache_dir='~/.energyflow')

In [76]:
val_data=energyflow.qg_jets.load(num_data=40000, pad=True, ncol=4, generator='pythia',with_bc=False, cache_dir='~/.energyflow')

In [77]:
val_p=val_data[0][:,:,1:3]
val_f=val_data[0]
val_m=(val_data[0][:,:,:]==0 ).astype(int)
val_m=val_m.sum(axis=2,keepdims=True)/4

In [79]:
data[1]

array([1., 1., 1., ..., 1., 0., 0.])

In [80]:
%cd /content/ParticleNet/tf-keras

/content/ParticleNet/tf-keras


In [81]:
p=data[0][:,:,1:3]
p.shape

(100000, 139, 2)

In [82]:
f=data[0]
f.shape

(100000, 139, 4)

In [83]:
m=(data[0][:,:,:]==0 ).astype(int)
m=m.sum(axis=2,keepdims=True)/4
# m=m.reshape(100000,1,139)
m.shape
# import numpy as np

# m=np.ones((200000, 139,1 ))

(100000, 139, 1)

In [84]:
train_dataset={}
train_dataset['points']=p
train_dataset['features']=f
train_dataset['mask']=m

In [85]:
import tensorflow as tf
from tensorflow import keras
from tf_keras_model import get_particle_net, get_particle_net_lite

In [86]:
model_type = 'particle_net_lite' # choose between 'particle_net' and 'particle_net_lite'
num_classes = 2
input_shapes = {k:train_dataset[k].shape[1:] for k in train_dataset}
if 'lite' in model_type:
    model = get_particle_net_lite(num_classes, input_shapes)
else:
    model = get_particle_net(num_classes,input_shapes)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [87]:
batch_size=1024

In [88]:
def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 10:
        lr *= 0.1
    elif epoch > 20:
        lr *= 0.01
    logging.info('Learning rate: %f'%lr)
    return lr

In [89]:
import logging
logging.basicConfig(level=logging.INFO, format='[%(asctime)s] %(levelname)s: %(message)s')

In [90]:
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(learning_rate=lr_schedule(0)),
              metrics=['accuracy'])
model.summary()

[2021-03-22 18:22:08,263] INFO: Learning rate: 0.001000


Model: "ParticleNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
mask (InputLayer)               [(None, 139, 1)]     0                                            
__________________________________________________________________________________________________
tf_op_layer_ParticleNet/NotEqua [(None, 139, 1)]     0           mask[0][0]                       
__________________________________________________________________________________________________
tf_op_layer_ParticleNet/Cast (T [(None, 139, 1)]     0           tf_op_layer_ParticleNet/NotEqual[
__________________________________________________________________________________________________
tf_op_layer_ParticleNet/Equal ( [(None, 139, 1)]     0           tf_op_layer_ParticleNet/Cast[0][0
________________________________________________________________________________________

In [92]:
import numpy as np
data1=data[1][:,np.newaxis]
data1.shape

(100000, 1)

In [93]:
from sklearn.preprocessing import OneHotEncoder


enc = OneHotEncoder()
enc.fit(data1)
out = enc.transform(data1).toarray()
out

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [1., 0.],
       [1., 0.]])

In [94]:

data2=val_data[1][:,np.newaxis]
enc2 = OneHotEncoder()
enc2.fit(data2)
out2 = enc.transform(data2).toarray()
out2.shape

(40000, 2)

In [95]:
out2

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [0., 1.]])

In [97]:

model.fit([p,f,m], out,
          batch_size=batch_size,
#           epochs=epochs,
          epochs=10,
          validation_data=([val_p,val_f,val_m], out2),
          shuffle=True,
          # callbacks=callbacks)
)

Train on 100000 samples, validate on 40000 samples
Epoch 1/10
100000/100000 [==============================] - 93s 928us/sample - loss: 0.5812 - acc: 0.7303 - val_loss: 0.7005 - val_acc: 0.5033
Epoch 2/10
100000/100000 [==============================] - 93s 926us/sample - loss: 0.5214 - acc: 0.7533 - val_loss: 0.5575 - val_acc: 0.7393
Epoch 3/10
100000/100000 [==============================] - 93s 925us/sample - loss: 0.5053 - acc: 0.7591 - val_loss: 0.4971 - val_acc: 0.7642
Epoch 4/10
100000/100000 [==============================] - 93s 927us/sample - loss: 0.5015 - acc: 0.7602 - val_loss: 0.5474 - val_acc: 0.7393
Epoch 5/10
100000/100000 [==============================] - 93s 926us/sample - loss: 0.5014 - acc: 0.7607 - val_loss: 0.4951 - val_acc: 0.7673
Epoch 6/10
100000/100000 [==============================] - 93s 926us/sample - loss: 0.4974 - acc: 0.7634 - val_loss: 0.4935 - val_acc: 0.7673
Epoch 7/10
100000/100000 [==============================] - 92s 923us/sample - loss: 0.4966

# ABC NET(modified GAPNet)

In [98]:
%cd /content

/content


In [99]:
!pwd

/content


In [100]:
! git clone https://github.com/shivamshan/ABCNet.git

fatal: destination path 'ABCNet' already exists and is not an empty directory.


In [101]:
!pip install energyflow

In [102]:
import energyflow

In [103]:
train_data=energyflow.qg_jets.load(num_data=100000, pad=True, ncol=4, generator='pythia',with_bc=False, cache_dir='~/.energyflow')

In [104]:
val_data=energyflow.qg_jets.load(num_data=40000, pad=True, ncol=4, generator='pythia',with_bc=False, cache_dir='~/.energyflow')

In [105]:
train_data[0][:,:,1:].shape

(100000, 139, 3)

In [106]:
train_data[1].shape

(100000,)

In [107]:
import numpy as np

In [108]:
train_file={}

train_file['data']=train_data[0][:,:,1:]

train_file['global']=train_data[0][:,:,0]

train_file['pid']=train_data[1]


In [109]:
test_file={}

test_file['data']=val_data[0][:,:,1:]

test_file['global']=val_data[0][:,:,0]

test_file['pid']=val_data[1]

In [110]:
%cd /content/ABCNet/classification/

/content/ABCNet/classification


In [111]:
s_train=np.sum(train_data[0],axis=1)
s_val=np.sum(val_data[0],axis=1)

In [112]:
g_train=s_train[:,0]
g_val=s_val[:,0]

In [113]:
import h5py

with h5py.File('train.hdf5', 'w') as f:  
  f.create_dataset('data',data=train_data[0][:,:,1:])
  f.create_dataset('global',data=g_train[:,np.newaxis])
  f.create_dataset('pid',data=train_data[1])


In [114]:
with h5py.File('test.hdf5', 'w') as f:  
  f.create_dataset('data',data=val_data[0][:,:,1:])
  f.create_dataset('global',data=g_val[:,np.newaxis])
  f.create_dataset('pid',data=val_data[1])

In [115]:
g_val[:,np.newaxis].shape

(40000, 1)

In [116]:
!pwd

/content/ABCNet/classification


In [117]:
%tensorflow_version 1.x

In [119]:
!python train.py --max_epoch 10 --num_glob 1 --num_point 139 --nfeat 3

Instructions for updating:
non-resource variables are not supported in the long term
pid: 2743
--- Get model and loss
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Instructions for updating:
keep_dims is deprecated, use keepdims instead
shape (64, 139, 1, 35)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
--- Get training operator
2021-03-22 18:42:05.687470: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2199995000 Hz
2021-03-22 18:42:05.687719: I tensorflow/compiler/xla/service/service

# **ACCURACY COMPARISON**


Note:- The following accuracies are on Validation/Test set after 10 epochs .

**Number of events used:** </br>
Training:- 100000 </br>
Test/Validaation:- 40000


*   # ParticleNET  --- 0.7677
*   # ABCNet --- 0.7766

